# 【案例11】无人机与货车合作配送——参考求解

In [1]:
import numpy as np
import pandas as pd
# 客户数量c
c = 5
# 生成节点的位置(index = 0是起始点，index = c+1是终止点,其他是客户节点)
# 假设所有的客户既可以由无人机配送，又可以用卡车配送
node_pos = np.random.rand(c+1, 2)
node_pos = np.vstack((node_pos, node_pos[0]))
print('node_pos:', node_pos)

# 将DataFrame保存到Excel文件
df = pd.DataFrame(node_pos, columns=['X', 'Y'])
df.to_excel('node_pos.xlsx', index=False)

node_pos: [[0.6685438  0.81848432]
 [0.09623973 0.73223414]
 [0.23265045 0.88822857]
 [0.49787731 0.82545236]
 [0.94464534 0.24991203]
 [0.15625954 0.14867157]
 [0.6685438  0.81848432]]


In [2]:
import math
###### Parameters ######
# C = {1,2,...,c} 代表客户
# C' = {1,2,..,c'} 能被无人机服务的客户
# 0--起始点，c+1:终止点

# N0 = {0,1,...,c}出发的节点i
# Nplus = {1,2,...,c+1} 到达的节点j

# i->j 卡车的运行时间t[i,j]
# i->j 无人机的运行时间t1[i,j]
# i = 0,1,...,c; j = 1,2,...,c+1
# t[i,i] ,t1[i,i] 不存在
# t[0,c+1] = 0
TRUCK_SPEED = 1
UAV_SPEED = 2
times = np.zeros([c+1,c+2])
times1 = np.zeros([c+1,c+2])
for i in range(0,c+1):
    for j in range(1,c+2):
        if i != j:
            d = math.sqrt( (node_pos[i][0]-node_pos[j][0])**2 + (node_pos[i][1]-node_pos[j][1])**2 )
            times[i][j] = d/TRUCK_SPEED
            times1[i][j] = d/UAV_SPEED 
times[0,c+1] = 0
print("times:",times)
print("times1:",times1)
        
# The prepare time to lauch UAV: sL 
sL = 0.1
# The recover time to recycle UAV: sR
sR = 0.1
# The flight endurance time of the UAV : e
e = 1

# <i,j,k> belongs to P when:
# i belongs to N0
# j belongs to C'
# k belongs to Nplus
# i!=j,i!=k,j!=k
# t1[i,j] + t1[j,k] <= e
belongsP = np.zeros([c+2,c+2,c+2])
for i in range(0,c+1):
    for j in range(1,c+1):
        for k in range(1,c+2):
            if (i!= j) and (j != k ) and (i != k) and (times1[i,j] + times1[j,k] <=e):
                belongsP[i][j][k] = 1
#print(belongsP)

times: [[0.         0.57876683 0.44143773 0.17080868 0.63206527 0.84325814
  0.        ]
 [0.         0.         0.20722487 0.41231345 0.97592351 0.58664099
  0.57876683]
 [0.         0.20722487 0.         0.27255483 0.95623466 0.74349185
  0.44143773]
 [0.         0.41231345 0.27255483 0.         0.7285934  0.75811275
  0.17080868]
 [0.         0.97592351 0.95623466 0.7285934  0.         0.79485961
  0.63206527]
 [0.         0.58664099 0.74349185 0.75811275 0.79485961 0.
  0.84325814]]
times1: [[0.         0.28938342 0.22071887 0.08540434 0.31603263 0.42162907
  0.        ]
 [0.         0.         0.10361243 0.20615673 0.48796175 0.2933205
  0.28938342]
 [0.         0.10361243 0.         0.13627742 0.47811733 0.37174592
  0.22071887]
 [0.         0.20615673 0.13627742 0.         0.3642967  0.37905638
  0.08540434]
 [0.         0.48796175 0.47811733 0.3642967  0.         0.3974298
  0.31603263]
 [0.         0.2933205  0.37174592 0.37905638 0.3974298  0.
  0.42162907]]


In [10]:
###### Decision Variables ######
# 定义线性规划问题
from pulp import *
prob = LpProblem("UAV", LpMinimize)

# x[i,j] (Binary) truck i->j ; i belongs to N0 ,j belongs to Nplus,i!=j
x = LpVariable.dicts('x',[(i,j) for i in range(0,c+1) for j in range(1,c+2) if i != j],cat='Binary')
# y[i,j,k] (Binary) UAV i->j->k .<i,j,k> belongs to P
y = LpVariable.dicts('y',[(i,j,k) for i in range(0,c+1) for j in range(1,c+1) for k in range(1,c+2) if (belongsP[i][j][k] == 1) and (j != i)],cat='Binary')
# t[j] the time truck arrives at j(j belongs to Nplus)
# t1[j] the time UAV arrives at j(j belongs to Nplus)
t = LpVariable.dicts('t',[j for j in range(0,c+2)],lowBound=0,cat='Continuous')
t1 = LpVariable.dicts('t1',[j for j in range(0,c+2)],lowBound=0,cat='Continuous')
######  Auxiliary decision variables ######
# p[i,j] (binary) the truck visit i before j(i != j) i,j belong to C
p = LpVariable.dicts('p',[(i,j)for i in range(0,c+1) for j in range(1,c+2) if i!=j ],cat='Binary')
# 1<=u[i]<=c+2 ,i belongs to Nplus
u = LpVariable.dicts('u',[i for i in range(1,c+2)],cat='Continuous')

    
prob += t[c+1]

# 约束条件
# （2）
for j in range(1,c+1):
    prob += lpSum(x[i,j] for i in range(0,c+1) if i != j) + lpSum(y[i,j,k] for i in range(0,c+1) for k in range(1,c+2) if belongsP[i][j][k] == 1) ==1

# (3)
prob += lpSum(x[0,j] for j in range(1,c+2)) == 1

# (4)
prob += lpSum(x[i,c+1] for i in range(0,c+1)) == 1

# (5)
for i in range(1,c+1):
    for j in range(1,c+2):
        if i== j :continue
        prob += u[i]-u[j]+1 <= (c+2)*(1-x[i,j])
# (6)
for j in range(1,c+1):
    prob += lpSum(x[i,j] for i in range(0,c+1) if i != j ) == lpSum(x[j,k] for k in range(1,c+2) if k != j)

#(7)
for i in range(0,c+1):
    prob += lpSum(y[i,j,k] for j in range(1,c+1) for k in range(1,c+2) if (j!=i)and(belongsP[i][j][k] == 1)) <= 1

# (8)
for k in range(1,c+2):
    prob += lpSum(y[i,j,k] for i in range(0,c+1) for j in range(1,c+1) if (i != k)and(belongsP[i][j][k] == 1) ) <= 1
    
# (9)
for i in range(1,c+1):
    for j in range(1,c+1):
        for k in range(1,c+2):
            if (j!= i) and (belongsP[i][j][k] == 1):
                prob += 2*y[i,j,k] <= lpSum(x[h,i] for h in range(0,c+1)if h != i) + lpSum(x[l,k] for l in range(1,c+1)if l!= k)

# (10)
for j in range(1,c+1):
    for k in range(1,c+2):
        if belongsP[0][j][k] == 1:
            prob += y[0,j,k] <= lpSum(x[h,k] for h in range(0,c+1) if h != k)
# (11)
for i in range(1,c+1):
    for k in range(1,c+2):
        if k!= i:
            prob += u[k] -u[i] >= 1-(c+2)*(1-lpSum(y[i,j,k] for j in range(1,c+1) if belongsP[i][j][k] == 1))
#(12)
M = 100000000
for i in range(1,c+1):
    prob += t1[i] >= t[i] - M * (1-lpSum(y[i,j,k] for j in range(1,c+1) for k in range(1,c+2) if (j!= i)and(belongsP[i][j][k] == 1)))

#(13)
for i in range(1,c+1):
    prob += t1[i] <= t[i] + M * (1-lpSum(y[i,j,k] for j in range(1,c+1) for k in range(1,c+2) if (j!= i)and(belongsP[i][j][k] == 1)))

# (14)
for k in range(1,c+2):
    prob += t1[k] >= t[k] - M * (1-lpSum(y[i,j,k] for i in range(0,c+1) for j in range(1,c+1) if (i!= k)and(belongsP[i][j][k] == 1)))

# (15)
for k in range(1,c+2):
    prob += t1[k] <= t[k] + M * (1-lpSum(y[i,j,k] for i in range(0,c+1) for j in range(1,c+1) if (i!= k)and(belongsP[i][j][k] == 1)))

# (16)
for h in range(0,c+1):
    for k in range(1,c+2):
        if k!=h:
            prob += t[k] >= t[h] + times[h][k] + sL*(lpSum(y[k,l,m] for l in range(1,c+1) for m in range(1,c+2) if (l != k)and(belongsP[k][l][m] == 1) )) + sR*lpSum(y[i,j,k] for i in range(0,c+1) for j in range(1,c+1) if (i!=k)and(belongsP[i][k][k] == 1)  ) - M *(1-x[h,k])

# (17)
for j in range(1,c+1):
    for i in range(0,c+1):
        if i!= j:
            prob += t1[j] >= t1[i] + times1[i][j] - M*(1-lpSum(y[i,j,k] for k in range(1,c+2) if belongsP[i][j][k] == 1))

# (18)AAA
for j in range(1,c+1):
    for k in range(1,c+2):
        if k!= j:
            prob += t1[k] >= t1[j] + times1[j][k] + sR - M*(1-lpSum(y[i,j,k] for i in range(0,c+1) if belongsP[i][j][k] == 1))

#(19) 
for k in range(1,c+2):
    for j in range(1,c+1):
        for i in range(0,c+1):
            if(j != k)and (belongsP[i][j][k] == 1) :
                prob += t1[k] - ( t1[j] - times1[i][j]) <= e + M*(1-y[i,j,k])

# (20)(21)(22)
for i in range(1,c+1):
    for j in range(1,c+1):
        if i != j:
            prob += u[i] - u[j] >= 1-(c+2)* p[i,j]
            prob += u[i] - u[j] <= -1 + (c+2) * (1-p[i,j])
            prob += p[i,j]+p[j,i] == 1

#(23)
for i in range(0,c+1):
    for k in range(1,c+2):
        for l in range(1,c+1):
            if (k!=i)and(l!=i)and(l!=k):
                prob += t1[l] >= t1[k] - M*(3-lpSum(y[i,j,k] for j in range(1,c+1) if (belongsP[i][j][k] == 1 )and(j != l ))- lpSum(y[l,m,n] for m in range(1,c+1) for n in range(1,c+2)if (m!=i)and(m!=k)and(m!=l)and(n!=i)and(n!=k)and(belongsP[l][m][n] == 1)) - p[i,l]  )

# （24）(25)
prob += t[0] == 0
prob += t1[0] == 0

#(26)
for j in range(1,c+1):
    prob += p[0,j] == 1
# （27）(28)略过
# （29）
for i in range(1,c+2):
    prob += u[i] <= c+2
    prob += u[i] >= 1

# (30)-(32)略

prob.solve()
if LpStatus[prob.status] == "Optimal":  
    print("找到最优解")
else:
    print("No solution")

找到最优解


In [14]:
# 打印配送方案
print("Time cost:",value(t[c+1]))
print(f"Start:{0},end:{c+1}")
for i in range(0,c+1):
    for j in range(1,c+2):
        if i!= j and value(x[i,j] ) == 1:
            print(f"Truck: {i}（{ value(t[i]):.2f}）->{j}（{value(t[j]):.2f}）")
for i in range(0,c+1):
    for j in range(1,c+1):
        for k in range(1,c+2):
            if (belongsP[i][j][k] == 1) and (j != i) and value(y[i,j,k]) == 1:
                print(f"UAV:{i}({value(t1[i]):.2f})->{j}({value(t1[j]):.2f})->{k}({value(t1[k]):.2f})")
                

Time cost: 1.7875301
Start:0,end:6
Truck: 0（0.00）->2（0.89）
Truck: 1（1.10）->3（1.62）
Truck: 2（0.89）->1（1.10）
Truck: 3（1.62）->6（1.79）
UAV:0(0.00)->5(0.42)->2(0.89)
UAV:2(0.89)->4(1.37)->6(1.79)


比较一下传统TSP问题得出的最佳时间：

In [15]:
# 传统TSP解法
TRUCK_SPEED = 1
timeCost = np.zeros([c+1,c+1])
for i in range(0,c+1):
    for j in range(0,c+1):
        d = math.sqrt( (node_pos[i][0]-node_pos[j][0])**2 + (node_pos[i][1]-node_pos[j][1])**2 )
        timeCost[i][j] = d/TRUCK_SPEED
# print("timeCost:\n",timeCost)

from pulp import *
import numpy as np

NUM_CITIES = c+1

# 创建问题
prob = LpProblem("TSP", LpMinimize)

# 创建二进制变量
# xij = 1 说明执行线路i->j
x = LpVariable.dicts('x', [(i, j) for i in range(1, NUM_CITIES + 1) for j in range(1, NUM_CITIES + 1) if i != j], cat='Binary')

# 定义目标函数
prob += lpSum([x[i, j] * timeCost[i-1][j-1] for i in range(1, NUM_CITIES + 1) for j in range(1, NUM_CITIES + 1) if i != j]) 

for i in range(1, NUM_CITIES + 1):
    prob += lpSum([x[(i, j)] for j in range(1, NUM_CITIES + 1) if i != j]) == 1
for j in range(1, NUM_CITIES + 1):
    prob += lpSum([x[(i, j)] for i in range(1, NUM_CITIES + 1) if i != j]) == 1
    
# MTZ约束条件,消除所有子回路，从而保证全局只有一条回路且该回路覆盖了所有节点。
u = LpVariable.dicts('u', range(NUM_CITIES), lowBound=1, upBound=NUM_CITIES, cat='Integer')
for i in range(1, NUM_CITIES):
    for j in range(1, NUM_CITIES):
        if i != j:
            prob += u[i] - u[j] + NUM_CITIES * x[(i, j)] <= NUM_CITIES - 1

# 求解问题
prob.solve()

# 打印结果
print("Total Time:", value(prob.objective))

current_city = 1
path = [1]
while True:
    next_city = [j for j in range(1, NUM_CITIES + 1) if (current_city != j)  and value(x[(current_city, j)]) == 1][0]
    if next_city == 1:
        path.append(1)
        break
    current_city = next_city
    path.append(current_city)
print("Path:",[i-1 for i in path])

Total Time: 2.6641542523811568
Path: [0, 4, 5, 1, 2, 3, 0]


当客户数量为5时，发现UAV与卡车合作配送的时间（t=1.78）比只用卡车配送的时间(t=2.66)要短，说明无人机和卡车的合作配送能提高整体的配送效率。